In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
# warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [3]:
train = pd.read_csv('../data/train_tweets.csv')
test = pd.read_csv('../data/test_tweets.csv')

In [30]:
data = pd.concat([train, test]).reset_index().drop('index', axis=1)

In [32]:
data.tail()

,id,label,tweet
49154,49155,NaN,thought factory: left-right polarisation! #tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...
49157,49158,NaN,"happy, at work conference: right mindset leads..."
49158,49159,NaN,"my song ""so glad"" free download! #shoegaze ..."
